<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d9c559a590db95273f11a8cba08c99c918a635abdb74d81b3efca1618bd2789a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-02 09:16:35
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -23.97 K (-0.17%)
Current PnL: -23.31 L (-15.28%)
CY Booked + Current PnL: -9.42 L (-6.17%)
-------------------
Total profit:  1.56 L
Total loss:  -24.87 L
-------------------
Total Booked + Current PnL: 17.56 L (13.98%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.75%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.72 L (62.27%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.08%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.60,-8.08,8.81,0.02,14136.0,-14103.0,160456.0,144.42,68.0,M-SC,13.11,234.0,-1.00,1.13,39.57,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.23,-16.62,20.05,0.09,16847.0,-16754.0,84023.0,95.09,37.0,M-SC,3.14,253.0,-0.99,0.59,10.39,OX40N,NTT,DURABLES
43,ITC,452.00,0.09,-1.14,11.72,10.44,23188.0,-2289.0,197849.0,-40.74,46.0,X-LC,1.37,5.0,-0.10,1.39,4.36,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.27,5.95,13.44,20.19,24823.0,10378.0,184698.0,-16.96,48.0,X-MC,6.96,68.0,0.42,1.30,21.97,X40,ATH,INSURANCE
83,VOLTAS,1530.00,-0.03,6.39,12.50,19.69,25500.0,12258.0,204000.0,-3.98,47.0,X-MC,3.13,78.0,0.48,1.43,13.62,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,1.96,-13.79,64.93,42.18,154721.0,-38121.0,238289.0,-23.08,36.0,X-LC,2.29,3.0,-0.25,1.67,3.22,XY24,NTT,AUTO
67,SIS,528.00,1.29,-24.91,61.49,21.26,51265.0,-27660.0,83372.0,1965.80,45.0,H-SC,3.85,166.0,-0.54,0.59,12.64,OX40N,NTT,MISC
40,INDUSINDBK,1800.00,0.81,-36.95,110.77,32.89,53921.0,-28528.0,48678.0,-701.24,62.0,L-MC,5.93,259.0,-0.53,0.34,34.06,XR,NTT,BANKS
81,VALIANTORG,838.00,0.72,-44.99,216.58,74.15,194918.0,-73607.0,89998.0,-262.20,38.0,H-SC,18.77,149.0,-0.38,0.63,15.36,XR,NTT,CHEMICALS
5,ASIANPAINT,3460.25,0.70,0.94,19.82,20.95,50368.0,2358.0,254126.0,-0.07,75.0,X-LC,16.42,31.0,0.05,1.78,36.91,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VAIBHAVGBL,521.00,-1.52,-26.70,119.32,60.77,159863.0,-48797.0,133978.0,54.27,42.0,H-SC,8.21,159.0,-0.31,0.94,20.66,XR,NTT,JEWELLERY
30,HATHWAY,31.40,-1.08,-29.36,145.31,73.29,111598.0,-31920.0,76800.0,791.30,33.0,H-SC,6.02,172.0,-0.29,0.54,2.73,XR,NTT,ENTERTAINMENT
16,CERA,9475.00,-1.06,-28.84,74.09,23.89,92744.0,-50725.0,125178.0,-31.26,29.0,H-SC,6.64,157.0,-0.55,0.88,8.11,OX40N,NTT,CERAMICS
65,SHALBY,327.00,-0.99,-21.56,59.48,25.10,84517.0,-39050.0,142093.0,944.66,27.0,M-SC,15.10,250.0,-0.46,1.00,13.87,XY24,NTT,HEALTHCARE
38,INDIAMART,4810.62,-0.97,-0.81,108.41,106.72,132623.0,-1001.0,122335.0,-50.82,37.0,H-SC,2.02,139.0,-0.01,0.86,22.38,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.97,-0.81,108.41,106.72,132623.0,-1001.0,122335.0,-50.82,37.0,H-SC,2.02,139.0,-0.01,0.86,22.38,AR,ATH,MISC
37,IEX,219.00,0.03,-0.41,49.23,48.63,98037.0,-814.0,199140.0,-33.63,63.0,H-SC,15.13,136.0,-0.01,1.40,11.10,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.16,-5.11,119.11,107.91,166873.0,-7544.0,140100.0,-22.98,35.0,M-SC,10.96,216.0,-0.05,0.98,0.18,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.60,-8.08,8.81,0.02,14136.0,-14103.0,160456.0,144.42,68.0,M-SC,13.11,234.0,-1.00,1.13,39.57,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.23,-16.62,20.05,0.09,16847.0,-16754.0,84023.0,95.09,37.0,M-SC,3.14,253.0,-0.99,0.59,10.39,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.61,-23.01,47.38,13.47,98371.0,-62046.0,207621.0,-68.31,32.0,H-SC,1.96,158.0,-0.63,1.46,5.57,XY24,NTT,PAINTS
67,SIS,528.00,1.29,-24.91,61.49,21.26,51265.0,-27660.0,83372.0,1965.80,45.0,H-SC,3.85,166.0,-0.54,0.59,12.64,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,0.13,2.79,67.59,72.27,117724.0,4726.0,174174.0,-11.98,61.0,M-LC,3.54,99.0,0.04,1.22,9.14,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.13,2.79,67.59,72.27,117724.0,4726.0,174174.0,-11.98,61.0,M-LC,3.54,99.0,0.04,1.22,9.14,XR,NTT,IT
38,INDIAMART,4810.62,-0.97,-0.81,108.41,106.72,132623.0,-1001.0,122335.0,-50.82,37.0,H-SC,2.02,139.0,-0.01,0.86,22.38,AR,ATH,MISC
37,IEX,219.00,0.03,-0.41,49.23,48.63,98037.0,-814.0,199140.0,-33.63,63.0,H-SC,15.13,136.0,-0.01,1.40,11.10,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.05,-1.30,37.41,35.63,77385.0,-2718.0,206856.0,-15.41,41.0,H-MC,3.68,119.0,-0.04,1.45,15.16,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.16,-5.11,119.11,107.91,166873.0,-7544.0,140100.0,-22.98,35.0,M-SC,10.96,216.0,-0.05,0.98,0.18,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.15,-37.26,115.52,35.20,93250.0,-47948.0,80722.0,5.85,19.0,X-SC,13.93,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,-0.53,-9.11,38.70,26.07,57593.0,-14920.0,148820.0,-33.02,27.0,X-MC,10.28,55.0,-0.26,1.04,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-0.55,-14.53,73.70,48.45,126295.0,-29142.0,171364.0,-20.73,28.0,X-MC,7.77,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
46,JSWINFRA,342.00,-0.22,-8.93,26.53,15.23,48405.0,-17895.0,182452.0,-22.11,30.0,X-MC,6.84,66.0,-0.37,1.28,20.44,X40N,NTT,REALTY
8,AWL,485.00,-0.39,-19.52,90.72,53.50,220319.0,-58895.0,242856.0,-61.08,32.0,X-MC,2.62,58.0,-0.27,1.70,7.87,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.09,-1.14,11.72,10.44,23188.0,-2289.0,197849.0,-40.74,46.0,X-LC,1.37,5.0,-0.10,1.39,4.36,X40,NTT,FMCG
3,ACC,3906.0,-0.13,-22.33,111.35,64.15,205786.0,-53141.0,184810.0,-54.57,48.0,X-SC,1.53,82.0,-0.26,1.30,3.67,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.77,1.62,22.80,24.79,58356.0,4092.0,255948.0,-3.02,48.0,X-SC,1.93,86.0,0.07,1.80,25.14,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.02,3.07,27.07,30.97,49310.0,5420.0,182158.0,-21.15,55.0,X-MC,2.18,75.0,0.11,1.28,16.01,X40,ATH,INSURANCE
41,INFY,2275.0,0.25,6.47,45.46,54.87,151442.0,20244.0,333132.0,-17.09,61.0,X-LC,2.29,2.0,0.13,2.34,15.29,X40,BTT,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.15,-37.26,115.52,35.20,93250.0,-47948.0,80722.0,5.85,19.0,X-SC,13.93,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.55,-14.53,73.70,48.45,126295.0,-29142.0,171364.0,-20.73,28.0,X-MC,7.77,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-0.53,-9.11,38.70,26.07,57593.0,-14920.0,148820.0,-33.02,27.0,X-MC,10.28,55.0,-0.26,1.04,0.00,X40,ATH,APPARELS
55,PGHH,17907.65,-0.13,-4.62,40.23,33.76,77061.0,-9270.0,191550.0,-32.78,34.0,X-MC,4.25,57.0,-0.12,1.34,0.34,X40,ATH,FMCG
56,QUESS,424.00,-0.76,-28.73,100.43,42.84,46528.0,-18677.0,46329.0,-53.17,40.0,X-SC,37.43,83.0,-0.40,0.33,0.81,XY24,NTT,MISC


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.76,-28.73,100.43,42.84,46528.0,-18677.0,46329.0,-53.17,40.0,X-SC,37.43,83.0,-0.40,0.33,0.81,XY24,NTT,MISC
60,RELAXO,1176.00,0.03,-47.20,188.45,52.31,144443.0,-68512.0,76648.0,-43.76,36.0,X-SC,4.90,91.0,-0.47,0.54,2.60,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.15,-37.26,115.52,35.20,93250.0,-47948.0,80722.0,5.85,19.0,X-SC,13.93,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.45,-6.88,30.56,21.58,39881.0,-9639.0,130500.0,-53.04,34.0,X-MC,6.00,56.0,-0.24,0.92,15.53,X40N,ATH,IT
52,MEDANTA,1486.00,-0.55,3.46,19.93,24.08,26177.0,4395.0,131345.0,-4.85,46.0,X-SC,5.86,89.0,0.17,0.92,23.76,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.19,-15.03,40.38,19.28,118703.0,-51990.0,293966.0,-25.89,56.0,X-LC,6.92,1.0,-0.44,2.06,8.67,X40,ATH,IT
41,INFY,2275.00,0.25,6.47,45.46,54.87,151442.0,20244.0,333132.0,-17.09,61.0,X-LC,2.29,2.0,0.13,2.34,15.29,X40,BTT,IT
76,TMPV,600.00,1.96,-13.79,64.93,42.18,154721.0,-38121.0,238289.0,-23.08,36.0,X-LC,2.29,3.0,-0.25,1.67,3.22,XY24,NTT,AUTO
82,VBL,671.64,0.54,-2.20,38.73,35.67,119632.0,-6954.0,308888.0,-13.89,63.0,X-LC,5.48,4.0,-0.06,2.17,11.35,X40N,ATH,FMCG
43,ITC,452.00,0.09,-1.14,11.72,10.44,23188.0,-2289.0,197849.0,-40.74,46.0,X-LC,1.37,5.0,-0.10,1.39,4.36,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.81,-36.95,110.77,32.89,53921.0,-28528.0,48678.0,-701.24,62.0,L-MC,5.93,259.0,-0.53,0.34,34.06,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.48,-15.69,115.48,81.67,91338.0,-14716.0,79094.0,7111.11,42.0,L-SC,18.98,271.0,-0.16,0.56,53.17,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.22,-11.80,58.21,39.54,51937.0,-11935.0,89224.0,-31.03,66.0,M-SC,6.65,220.0,-0.23,0.63,25.81,AR,ATH,AUTO
51,MASFIN,398.61,-0.91,-4.62,27.96,22.05,26129.0,-4530.0,93450.0,-17.80,52.0,H-SC,6.50,164.0,-0.17,0.66,35.84,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.67,-19.48,52.79,23.03,76122.0,-34874.0,144197.0,57.89,49.0,H-SC,9.83,167.0,-0.46,1.01,43.07,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.60,-8.08,8.81,0.02,14136.0,-14103.0,160456.0,144.42,68.0,M-SC,13.11,234.0,-1.00,1.13,39.57,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.70,0.94,19.82,20.95,50368.0,2358.0,254126.0,-0.07,75.0,X-LC,16.42,31.0,0.05,1.78,36.91,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.81,-36.95,110.77,32.89,53921.0,-28528.0,48678.0,-701.24,62.0,L-MC,5.93,259.0,-0.53,0.34,34.06,XR,NTT,BANKS
50,LTIM,7201.88,-0.77,11.61,17.97,31.66,49368.0,28574.0,274725.0,7.45,65.0,H-LC,14.41,43.0,0.58,1.93,52.43,X200,ATH,IT
32,HCLTECH,1922.01,0.04,6.66,16.95,24.73,43736.0,16106.0,258030.0,11.49,66.0,X-LC,5.02,7.0,0.37,1.81,22.91,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.95
1,25,44.86
2,50,75.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.38
MC    29.10
LC    25.52
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.70
X40      21.25
X40N     12.05
XR        9.60
XY25      9.08
AR        9.06
OX40N     7.61
X200      1.93
SR        0.97
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.87
X-MC    22.63
X-LC    20.23
M-SC    11.87
X-SC     8.25
H-MC     4.76
H-LC     3.09
L-SC     1.39
M-MC     1.37
M-LC     1.22
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.83,-4.86,38.78
IT,13.87,-14.40,73.47
FINANCE,9.46,-15.05,63.66
MISC,7.42,-23.21,73.68
ELECTRICAL,6.02,-9.48,49.59
PAINTS,5.95,-8.93,25.11
INSURANCE,4.42,-0.49,35.90
PHARMA,3.89,-0.64,32.85
AUTO,3.40,-18.59,69.45


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3089237.0,21
XR,1295960.0,13
AR,1281370.0,10
X40,1008954.0,14
X40N,732846.0,9
OX40N,709886.0,10
XY25,345966.0,6
SR,281396.0,2
MH,77381.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3579735.0,25
M-SC,1369763.0,15
X-MC,1255988.0,16
X-LC,878701.0,11
X-SC,746507.0,8
H-MC,400162.0,3
L-SC,263751.0,3
M-LC,117724.0,1
H-LC,117089.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1226635.0      6
           AR         883714.0      5
M-SC       XY24       789328.0      6
H-SC       XR         780600.0      7
X-MC       X40        447734.0      7
X-LC       X40        387437.0      5
X-MC       XY24       377558.0      3
H-SC       OX40N      330009.0      4
X-SC       X40N       294233.0      3
M-SC       OX40N      292248.0      5
H-SC       SR         281396.0      2
X-SC       XY24       278491.0      3
X-MC       X40N       247797.0      4
X-LC       XY24       226404.0      2
H-MC       AR         209341.0      2
           XY24       190821.0      1
X-LC       X40N       190816.0      2
X-MC       XY25       182899.0      2
L-SC       XR         176122.0      2
X-SC       X40        173783.0      2
M-SC       XR         167593.0      2
           AR         120594.0      2
M-LC       XR         117724.0      1
L-SC       OX40N       87629.0      1
H-SC       MH          77381.0      1
X-LC       XY25        74044.0      2
H-LC       AR          67721.0      1
L-MC       XR          53921.0      1
M-MC       XY25        49716.0      1
H-LC       X200        49368.0      1
L-LC       XY25        39307.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
